In [3]:
! pwd

/home/arunava/VQA_Med


In [1]:
path = '/home/arunava'
#****************************To be Change to reproduce ou result*********************************************

# location of the data and where to store iamge feature image
path_output_chd = path+'/VQA_Med'    

input_vqa_train = 'train_dataset_pickle/train19_subset20_21_df.pkl'
input_vqa_valid ='valid_dataset_pickle/val19_df.pkl'

img_feat_train = 'train_dataset_pickle/train-image-feature-19.pickle'
img_feat_valid ='valid_dataset_pickle/valid-image-feature-19.pickle'

input_test = 'test_dataset_pickle/C1_test_dataset_df.pkl'
img_feat_test = 'test_dataset_pickle/test-image-feature.pickle'

In [2]:
import json
with open(path+'/VQA_Med/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

178


In [3]:
import easydict
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 2048,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":200,
    })

In [4]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import re
import collections
import pickle
import os
from tqdm import tqdm
import json
import torch.nn as nn
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision
from PIL import Image
import torch.nn.functional as f
import random
from sklearn.preprocessing import LabelEncoder

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
category_names = {'C1': 'Modality','C2': 'Plane','C3': 'Organ', 'C4': 'Abnormality','All': 'All dataset',}

In [7]:
path_change = path
seed_value = opt.SEED
print(seed_value) # 97
np.random.seed(seed_value) # return None
random.seed(seed_value) # return None
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) # return None
torch.cuda.manual_seed_all(seed_value) # return None
torch.backends.cudnn.enabled = False 
''' backends.cudnn.enabled enables cudnn for some operations such as conv layers and RNNs, which can yield 
a significant speedup. The cudnn RNN implementation doesn’t support the backward operation during eval() 
and thus raises the error. You could disable cudnn for your workload (as already done) or try to call .train()
on the RNN module separately after using model.eval().'''
torch.backends.cudnn.benchmark = False
'''If your model does not change and your input sizes remain the same - then you may benefit from setting 
torch.backends.cudnn.benchmark = True.However, if your model changes: for instance, if you have layers that
are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a 
different number of times, then setting torch.backends.cudnn.benchmark = True might stall your execution.'''
torch.backends.cudnn.deterministic = False
'''torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. 
Therefore, no, it will not guarantee that your training process is deterministic, since you're also using
torch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA.'''

97


"torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. \nTherefore, no, it will not guarantee that your training process is deterministic, since you're also using\ntorch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA."

In [11]:
#set path for the dataset
train_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/'#QAPairsByCategory/'
train_images_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/Train_images/'

valid_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/'#QAPairsByCategory/'
valid_images_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/Val_images/'

test_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/' 
test_images_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/VQAMed2019_Test_Images/'

size = opt.IMG_INPUT_SIZE
size

224

In [12]:
class ResNet50(nn.Module):
    def __init__(self,patch_size=4):

        super(ResNet50,self).__init__()
        resnet_model = torchvision.models.resnet50(pretrained=True)
        # vgg19_model = torchvision.models.vgg19(pretrained=True)
        # print('resnet_model',resnet_model)
        # print('vgg19_model',vgg19_model)
        # print('list',list(resnet_model.children())[8])
        # cnt=0
        # self.children_list=[]
        # for n,c in resnet_model.named_children():
        #         self.children_list.append(c)
        #         if n == 'layer4':
        #             break

        # self.net = nn.Sequential(*self.children_list)

        self.img_feature_extractor = nn.Sequential(*(list(resnet_model.children())[:-2]))
        self.resize_dim = nn.AdaptiveAvgPool2d((patch_size,patch_size))
        # self.Conv1 = nn.Sequential(*list(resnet_model.children())[0:4])
        # self.Conv2 = nn.Sequential(*list(resnet_model.children())[4:7])
        # self.Conv3 = nn.Sequential(*list(resnet_model.children())[7:11])
        # self.Conv4 = nn.Sequential(*list(resnet_model.children())[11:17])
        # self.Conv5 = nn.Sequential(*list(resnet_model.children())[17:20])
        # self.avgpool = nn.Sequential(list(resnet_model.children())[8]) 

        
    def forward(self,image):

        with torch.no_grad():
            image_feat = self.resize_dim(self.img_feature_extractor(image))
           
        
        image_feat = image_feat.mean([2,3],keepdim=True)
        image_feat = torch.cat([ image_feat ], 1)
        # print('image_feat',image_feat.shape)
        l2_norm = image_feat.norm(p=2, dim=1, keepdim=True).detach()
        image_feat = image_feat.div(l2_norm)
        batch_size = image_feat.shape[0]
        embedding_dim_size = image_feat.shape[1]
        image_feature = image_feat.view(batch_size, embedding_dim_size, -1).squeeze(0)
        # print('image_feature.shape',image_feature.shape)
        return image_feature

In [13]:
# #Extract image feature
# class VGG19(nn.Module):
#     def __init__(self):
#         '''
#              We remove all the fully-connected layers in the VGG19 network and the convolution outputs of different feature scales
#                 are concatenated after global average pooling and l2-norm to form a 1984-dimensional vector to represent the image
#         '''
#         super(VGG19,self).__init__()
#         vgg_model = torchvision.models.vgg19(pretrained=True)	

#         self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
#         self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9])
#         self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
#         self.Conv4 = nn.Sequential(*list(vgg_model.features.children())[16:23])
#         self.Conv5 = nn.Sequential(*list(vgg_model.features.children())[23:30])
#         self.Conv6 = nn.Sequential(*list(vgg_model.features.children())[30:36])

#         self.avgpool = nn.Sequential(list(vgg_model.children())[1])
    
#     def forward(self,image):

#         with torch.no_grad():
#             out1 = self.Conv1(image)
#             out2 = self.Conv2(out1)
#             out3 = self.Conv3(out2)
#             out4 = self.Conv4(out3)          
#             out5 = self.Conv5(out4)          # [N, 512, 14, 14]
#             out6 = self.Conv6(out5) 

#             out7 = self.avgpool(out6)

#         #global average pooling
#         out1 = out1.mean([2,3],keepdim=True)
#         out2 = out2.mean([2,3],keepdim=True)
#         out3 = out3.mean([2,3],keepdim=True)
#         out4 = out4.mean([2,3],keepdim=True)
#         out5 = out5.mean([2,3],keepdim=True)
#         out6 = out6.mean([2,3],keepdim=True)
#         out7 = out7.mean([2,3],keepdim=True)

       
#         concat_features = torch.cat([out1, out2, out3, out4,out5, out6 ], 1) 

#         #l2-normalized feature vector
#         l2_norm = concat_features.norm(p=2, dim=1, keepdim=True).detach() 
#         concat_features = concat_features.div(l2_norm)               # l2-normalized feature vector
       

#         batch_size = concat_features.shape[0]
#         embedding_dim_size = concat_features.shape[1]
#         image_feature = concat_features.view(batch_size, embedding_dim_size, -1).squeeze(0) # [N, 1984, 1]


#         return image_feature

In [14]:
# vgg19_model = VGG19().to(device)

In [15]:
resnet50_model = ResNet50().to(device)

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
! pip install torchinfo

In [17]:
from torchinfo import summary
print(summary(resnet50_model))

Layer (type:depth-idx)                        Param #
ResNet50                                      --
├─Sequential: 1-1                             --
│    └─Conv2d: 2-1                            9,408
│    └─BatchNorm2d: 2-2                       128
│    └─ReLU: 2-3                              --
│    └─MaxPool2d: 2-4                         --
│    └─Sequential: 2-5                        --
│    │    └─Bottleneck: 3-1                   75,008
│    │    └─Bottleneck: 3-2                   70,400
│    │    └─Bottleneck: 3-3                   70,400
│    └─Sequential: 2-6                        --
│    │    └─Bottleneck: 3-4                   379,392
│    │    └─Bottleneck: 3-5                   280,064
│    │    └─Bottleneck: 3-6                   280,064
│    │    └─Bottleneck: 3-7                   280,064
│    └─Sequential: 2-7                        --
│    │    └─Bottleneck: 3-8                   1,512,448
│    │    └─Bottleneck: 3-9                   1,117,184
│    │    └─Bo

In [18]:
#transformation we apply to each image
# transform_
transform = {
    phase: transforms.Compose([transforms.RandomResizedCrop(size), 
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))]) 

    for phase in ['train', 'valid']}

test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),
                                                    (0.229, 0.224, 0.225))])
    

In [19]:
def count_answer_freq(df_data):
    '''
    count the frequence of each unique answer on the dataset
    '''
    all_answers = df_data['Answer'].values
    answer_freq_dict = defaultdict(int)
    for answer in all_answers:
        answer_freq_dict[answer] += 1
    answer_freq_dict_sort = dict(sorted(answer_freq_dict.items(), key=lambda x: x[1], reverse=True))

    return answer_freq_dict_sort

In [20]:

def get_most_frequent_classes(answer_freq_dict, threshold=1):
    print('threshold',threshold)
    final_classes = defaultdict(int)
    index = 0

    for answer,ans_freq in answer_freq_dict.items():

        if ans_freq >= threshold:
            final_classes[answer]= index         
        else:
            final_classes['UNKNOWN'] = index
            break
        index += 1  
    final_classes['UNKNOWN'] = index
    with open('answer_classes.json', 'w') as fp:
        json.dump(final_classes, fp)
    return final_classes

In [21]:
def parse_sentence(s):
    '''remove some character on the question'''
    s = s.replace(" - ", "-")
    s = s.lower()
    s = re.sub("\s\s+", " ", s)
    return s

In [22]:
#read a txt train file  and structure it in a dataframe
def get_category_file_train(category_name, category_path ,images_path, transform=None):
    
    data = []
    
    dict_data = { 'image_id':[],
                    'image_path':[],
                'Question':[],
                'Answer':[],
              }
    image_feat  = { }
    with open(category_path) as f:
        lines = f.readlines()
    cnt=0
    for element in lines:
        pd_element = element.split('|')
        dict_data['image_id'].append(pd_element[0])
        dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
        dict_data['Answer'].append(pd_element[2].strip("\n")) 
        
        image = Image.open(images_path+pd_element[0]+'.jpg').convert('RGB')
        # print('images',type(image))
        if transform:
            image = transform(image)

        dict_data['image_path'].append(images_path+pd_element[0]+'.jpg')
        image_feature = resnet50_model(image[None,...].to(device))
        image_feat[pd_element[0]] = image_feature.cpu().numpy()

        # cnt = cnt+1
        # if (cnt > 10): break
    
    df_data = pd.DataFrame(dict_data, columns = ['image_id',
                                                    'image_path',
                                                 'Question',
                                                 'Answer',
                                                 ])


    answer_freq = count_answer_freq(df_data)
    
    classes = get_most_frequent_classes(answer_freq, threshold=5)
    
    df_data['labels'] = df_data['Answer'].apply(lambda x : classes[x] if x in classes else classes['UNKNOWN'])
    
    return df_data, classes , image_feat


In [23]:
train_dataset_df, classes, image_feat_train = get_category_file_train(category_name=category_names['All'],
                                                    category_path=train_path+'All_QA_Pairs_train.txt', 
                                                    images_path=train_images_path,
                                                    transform=transform['train'])



threshold 5


In [24]:
train_dataset_df

image_id                                         image_path  \
0      synpic41148  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
1      synpic43984  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
2      synpic38930  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
3      synpic52143  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
4      synpic20934  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
...            ...                                                ...   
12787  synpic35685  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
12788  synpic35687  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
12789  synpic35701  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
12790  synpic48036  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   
12791  synpic54897  /home/arunava/VQA_Med_2019_Dataset/Train/Image...   

                                                Question  \
0                            what kind of image is this?   
1                           is this a t1 weighted image?   
2      what type of imaging modality is used to acqui...   
3                             is this a noncontrast mri?   
4                   what type of image modality is this?   
...                                                  ...   
12787                              is this image normal?   
12788             is there something wrong in the image?   
12789            is there evidence of any abnormalities?   
12790                             is the ct scan normal?   
12791                is there an abnormality in the mri?   

                     Answer  labels  
0      cta - ct angiography      32  
1                        no       2  
2           us - ultrasound      15  
3                        no       2  
4           xr - plain film       5  
...                     ...     ...  
12787                   yes       3  
12788                    no       2  
12789                    no       2  
12790                   yes       3  
12791                    no       2  

[12792 rows x 5 columns]

In [25]:
pickle_name='train_dataset_df'
train_dataset_df.to_pickle(path_output_chd+'/train_dataset_pickle/'+pickle_name+'.pkl')

In [26]:
with open(path_output_chd+'/train_dataset_pickle/train-image-feature.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat_train, f, pickle.HIGHEST_PROTOCOL)

In [27]:
 #read a txt validation file and structure it in a dataframe
def get_category_file_valid(category_name, category_path ,images_path, classes, transform=None):
    
    data = []
    
    dict_data = { 'image_id':[],
                    'image_path':[],
                'Question':[],
                'Answer':[],
              }
    image_feat = { }
    with open(category_path) as f:
        lines = f.readlines()
    
    for element in lines:
        pd_element = element.split('|')
        dict_data['image_id'].append(pd_element[0])
        dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
        dict_data['Answer'].append(pd_element[2].strip("\n")) 
        image = Image.open(images_path+pd_element[0]+'.jpg').convert('RGB')
        if transform:
            image = transform(image)
        dict_data['image_path'].append(images_path+pd_element[0]+'.jpg')
        image_feature = resnet50_model(image[None,...].to(device))
        image_feat[pd_element[0]] = image_feature.cpu().numpy()
        
    df_data = pd.DataFrame(dict_data, columns = ['image_id',
                                                    'image_path', 
                                                 'Question',
                                                 'Answer',
                                                 ])
    
    df_data['labels'] = df_data['Answer'].apply(lambda x : classes[x] if x in classes else classes['UNKNOWN'])

    
    return df_data,image_feat 

In [28]:
valid_dataset_df,image_feat_valid = get_category_file_valid(category_name=category_names['All'],
                                                    category_path=valid_path+ 'All_QA_Pairs_val.txt',
                                                    images_path=valid_images_path,
                                                    classes= classes,
                                                    transform=transform['valid'])


In [29]:
valid_dataset_df

image_id                                         image_path  \
0     synpic54733  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1     synpic25647  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
2     synpic35681  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
3     synpic39641  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
4     synpic35693  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
...           ...                                                ...   
1995  synpic45828  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1996  synpic60458  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1997  synpic43524  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1998  synpic59568  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1999  synpic48459  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   

                                               Question  \
0     what imaging modality was used to take this im...   
1                            what kind of scan is this?   
2                      was the mri taken with contrast?   
3     is this a t1 weighted, t2 weighted, or flair i...   
4                            is this a noncontrast mri?   
...                                                 ...   
1995                       what is abnormal in the mri?   
1996                     what is abnormal in the x-ray?   
1997              what is most alarming about this mri?   
1998                       what is abnormal in the mri?   
1999              what is most alarming about this mri?   

                                               Answer  labels  
0                                 ct with iv contrast      33  
1                                     xr - plain film       5  
2                                                  no       2  
3                                                  t2      12  
4                                                 yes       3  
...                                               ...     ...  
1995  marked hydrocephalus due to aqueductal stenosis     177  
1996                                 discoid meniscus     177  
1997    choroid plexus neoplasm, papilloma, carcinoma     127  
1998                               multiple sclerosis      62  
1999                           spine,  tethered  cord     177  

[2000 rows x 5 columns]

In [30]:
pickle_name='valid_dataset_df'
valid_dataset_df.to_pickle(path_output_chd+'/valid_dataset_pickle/'+pickle_name+'.pkl')

In [31]:
with open(path_output_chd+'/valid_dataset_pickle/valid-image-feature.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat_valid, f, pickle.HIGHEST_PROTOCOL)

In [32]:
#read a txt test file and structure it in a dataframe
def get_test_file(category_name, category_path ,images_path, transform=None,group='test'):
    
    data = []
    
    dict_data = { 'image_id':[],
                    'image_path':[],
                'Question':[],
                'Group':[],
              }

    image_feat = { }
    with open(category_path) as f:
        lines = f.readlines()
    
    # cnt=0;
    for element in lines:
        pd_element = element.split('|')
        dict_data['image_id'].append(pd_element[0])
        dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip("\n")) 
        dict_data['Group'].append(group)
        image = Image.open(images_path+pd_element[0]+'.jpg').convert('RGB')
        
        if transform:
            image = transform(image)
        # print('image.shape',image.shape)
        # dict_data['image'].append(image[None,...])
        dict_data['image_path'].append(images_path+pd_element[0]+'.jpg')
        image_feature = resnet50_model(image[None,...].to(device))
        image_feat[pd_element[0]] = image_feature.cpu().numpy()
        # print('cnt',cnt)
        # cnt=cnt+1
        
    df_data = pd.DataFrame(dict_data, columns = ['image_id',
                                                    'image_path',
                                                 'Question',
                                                 'Group', 
                                                 ])
    return df_data,image_feat

In [33]:
test_dataset_df,image_feat_test = get_test_file(category_name=category_names['All'], 
                                                  category_path=test_path+'VQAMed2019_Test_Questions.txt'  ,
                                                  images_path=test_images_path,
                                                  transform=test_transform,
                                                  group='test')
# C1_test_dataset_df= test_dataset_df

In [34]:
test_dataset_df

image_id                                         image_path  \
0    synpic54082  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
1    synpic48556  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
2    synpic50696  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
3    synpic37194  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
4    synpic31308  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
..           ...                                                ...   
495  synpic21789  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
496  synpic39878  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
497  synpic41525  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
498  synpic18173  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
499  synpic54143  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   

                                            Question Group  
0                           what modality is shown?   test  
1                  was the mri taken with contrast?   test  
2      what type of contrast did this patient have?   test  
3                     what imaging method was used?   test  
4                           what modality is shown?   test  
..                                               ...   ...  
495                what is abnormal in the ct scan?   test  
496           what is most alarming about this mri?   test  
497       what is most alarming about this ct scan?   test  
498  what is the primary abnormality in this image?   test  
499  what is the primary abnormality in this image?   test  

[500 rows x 4 columns]

In [35]:
pickle_name='test_dataset_df'
test_dataset_df.to_pickle(path_output_chd+'/test_dataset_pickle/'+pickle_name+'.pkl')

In [36]:
with open(path_output_chd+'/test_dataset_pickle/test-image-feature.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat_test, f, pickle.HIGHEST_PROTOCOL)

In [37]:
class VqaDataset(data.Dataset):
    '''
        Main class use to retrieve our dataset from pickle file.
    '''

    def __init__(self, input_vqa, img_feat_vqa, transform=None, phase = 'train'):
        # self.input_dir = input_dir
        # self.vqa = np.load(input_dir+'/'+input_vqa, allow_pickle=True )
        # self.img_feat_vqa = np.load(input_dir+'/'+img_feat_vqa, allow_pickle=True )
        self.vqa = input_vqa
        self.vocab_size = None
        self.phase = phase
        self.transform = transform
   

    def __getitem__(self, idx):

        vqa = self.vqa
        phase = self.phase
        transform=self.transform[phase]
        # img_feat_vqa = self.img_feat_vqa
        
        image_id = vqa['image_id'].values[idx]
        image_path = vqa['image_path'].values[idx]
        # image_feat = torch.Tensor(img_feat_vqa[image_id])
        # print('image_path',image_path)
        
        image = Image.open(image_path).convert('RGB')
        # print('transform',transform)
        if transform:
            image = transform(image)

        qst2idc =  vqa['Question'].values[idx]
        sample = { 'question': qst2idc, 'image': image, 'image_id':image_id } 
        if (self.phase == 'train') or  (self.phase == 'valid'):
            ans2idc = vqa['labels'].values[idx]
            answer_text = vqa['Answer'].values[idx]
            sample['image_id'] = image_id
            sample['label'] = ans2idc
            sample['answer_text'] = answer_text
        else:
            sample['image_id'] = image_id
            
            
        return sample

    def __len__(self):

        return len(self.vqa)


def get_loader(input_vqa_train, input_vqa_valid, batch_size, num_workers, transform=None,size=228):
    '''
        Load our dataset with dataloader for the train and valid data
    '''

    vqa_dataset = {
        'train': VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_vqa_train,
            img_feat_vqa=img_feat_train,
            transform = transform,
            phase = 'train'),
        'valid': VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_vqa_valid,
            img_feat_vqa=img_feat_valid,
            transform = transform,
            phase = 'valid')}
    

    data_loader = {
        phase: torch.utils.data.DataLoader(
            dataset=vqa_dataset[phase],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            )
        for phase in ['train','valid']}

    return data_loader


def get_test_loader(input_test, batch_size, num_workers,size=228):

   
    
    test_vqa_dataset = VqaDataset(
            # input_dir=input_dir,
            input_vqa=input_test,
            # img_feat_vqa=img_feat_vqa,
            phase = 'test')
    
    
    data_loader = torch.utils.data.DataLoader(dataset=test_vqa_dataset,
                                                transform=transform,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                num_workers=num_workers)
    return data_loader

In [38]:
input_vqa_train = train_dataset_df
input_vqa_valid = valid_dataset_df

saved_dir = path_output_chd

num_epochs = opt.NUM_EPOCHS
image_size = opt.IMG_INPUT_SIZE
num_workers = 0
batch_size = opt.BATCH_SIZE

In [39]:
data_loader = get_loader(
        input_vqa_train = train_dataset_df, 
        input_vqa_valid = input_vqa_valid,
        batch_size = batch_size, 
        num_workers = num_workers,
        transform = transform,
        size = image_size )

In [40]:
le = LabelEncoder()

In [41]:
def mixup_data_vqa(id,v, q, a, al, alpha=1.0, use_cuda=True):
    '''Returns mixed images, pairs of questions, targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = v.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    # print('index',index)

    mixed_v = lam * v + (1 - lam) * v[index, :]
    # print('mixed_v',mixed_v.shape)
    id_a, id_b = id,id[index]
    a_a, a_b = a, a[index]
    al_a,al_b = al,al[index]
    # print('a_a',a_a)
    # print('a_b',a_b)
    q_a, q_b = q, q[index]
    # print('q_a',q_a)
    # print('q_b',q_b)
    
    return id_a,id_b,mixed_v, a_a, a_b,al_a,al_b, q_a, q_b, lam

In [42]:
image_feat = {}
dict_data={'image_id': [], 'mixed_image': [],  'question_a': [], 'question_b':[], 'labels_a':[] , 'labels_b':[], 'answer_a' : [], 'answer_b':[], 'lam':[]  }

In [43]:
for batch_idx, batch_sample in enumerate(data_loader['train']):
    image_id = batch_sample['image_id']
    image = batch_sample['image'].to(device)
    questions = batch_sample['question']
    labels = batch_sample['label'].to(device)
    label_answer_text = batch_sample['answer_text']
    le1 = LabelEncoder()
    label_answers_text_l = le1.fit_transform(label_answer_text)
    label_answers_text_l = torch.as_tensor(label_answers_text_l).to(device)
    le2 = LabelEncoder() 
    question_labels = le2.fit_transform(questions)
    question_labels = torch.as_tensor(question_labels).to(device)
    le3 = LabelEncoder()
    image_id_labels = le3.fit_transform(image_id)
    image_id_labels = torch.as_tensor(image_id_labels).to(device)



    id_a,id_b,mixed_v, a_a, a_b,al_a,al_b, q_a, q_b, lam = mixup_data_vqa(
                image_id_labels,image, question_labels, labels, label_answers_text_l, alpha=0.4, use_cuda=True)

    # mixed_v = mixed_v.cpu().data.numpy()
    a_a = a_a.cpu().data.numpy()
    a_b = a_b.cpu().data.numpy()
    id_a = id_a.cpu().data.numpy()
    id_b = id_b.cpu().data.numpy()
    q_a = q_a.cpu().data.numpy()
    q_b = q_b.cpu().data.numpy()
    al_a = al_a.cpu().data.numpy() 
    al_b = al_b.cpu().data.numpy()
    id_a_l = le3.inverse_transform(id_a)
    id_b_l = le3.inverse_transform(id_b)
    q_a_l = le2.inverse_transform(q_a)
    q_b_l = le2.inverse_transform(q_b)
    al_a_l = le1.inverse_transform(al_a)
    al_b_l = le1.inverse_transform(al_b)

    for index,img in enumerate(mixed_v):
        # print('img_shape',img.shape)
        mix_image_feature = resnet50_model(img[None,...]).to(device)
        # print('mix_img_feature.shape',mix_image_feature.shape)
        image_feat[id_a_l[index]+'_'+id_b_l[index]] = mix_image_feature.cpu().numpy()



    mixed_v = mixed_v.cpu().data.numpy()
    
    for index in range(len(mixed_v)):
        # print(id_a[index])
        dict_data['image_id'].append(id_a_l[index]+'_'+id_b_l[index])
        # dict_data['image_id'].append(id_a_l[index]+'_'+id_b_l[index])
        dict_data['mixed_image'].append(mixed_v[index])
        # dict_data['mixed_image'].append(mixed_v[index])
        dict_data['question_a'].append(q_a_l[index])
        dict_data['question_b'].append(q_b_l[index])
        dict_data['labels_a'].append(a_a[index])
        dict_data['labels_b'].append(a_b[index])
        dict_data['answer_a'].append(al_a_l[index])
        dict_data['answer_b'].append(al_b_l[index])
        dict_data['lamda'] = lam



In [44]:
mixed_train_df_data = pd.DataFrame(dict_data, columns = ['image_id',
                                                    'mixed_image', 
                                                    'question_a',
                                                    'question_b',
                                                    'answer_a',
                                                    'answer_b',
                                                    'labels_a',
                                                    'labels_b',
                                                    'lamda'
                                                ])

In [45]:
mixed_train_df_data

image_id  \
0       synpic33704_synpic32688   
1       synpic36106_synpic31209   
2       synpic39327_synpic55059   
3       synpic31209_synpic33704   
4       synpic19890_synpic34146   
...                         ...   
12787  synpic46591_synpic100514   
12788   synpic48578_synpic33843   
12789   synpic55583_synpic24141   
12790   synpic50342_synpic40922   
12791   synpic41294_synpic31356   

                                             mixed_image  \
0      [[[-1.361599, -1.3391032, -1.3546426, -1.39228...   
1      [[[-1.2561179, -1.2035546, -1.1171383, -1.0524...   
2      [[[-1.5742296, -1.5658653, -1.5449548, -1.5324...   
3      [[[-1.8888996, -1.9498268, -1.9502233, -1.9502...   
4      [[[-1.8809315, -1.9448521, -1.8668001, -1.8020...   
...                                                  ...   
12787  [[[-0.92743194, -0.9411576, -0.9823345, -1.023...   
12788  [[[-2.012531, -1.9989345, -1.985338, -1.975140...   
12789  [[[1.3678015, 1.2821777, 1.306488, 1.2964199, ...   
12790  [[[-1.610401, -1.6138002, -1.6138002, -1.61040...   
12791  [[[-1.4078288, -1.5450851, -1.6377654, -1.6994...   

                                           question_a  \
0                       which organ system is imaged?   
1                          in what plane is this mri?   
2                    in what plane is this mri taken?   
3                 what is the modality of this image?   
4                               is this an mri image?   
...                                               ...   
12787  what is the primary abnormality in this image?   
12788                    what is abnormal in the mri?   
12789                     what plane is demonstrated?   
12790              in what plane is this image taken?   
12791                           is this an mri image?   

                                   question_b  \
0       in what plane is this x-ray captured?   
1         what is the modality of this image?   
2                 what kind of image is this?   
3               which organ system is imaged?   
4        the ct scan shows what organ system?   
...                                       ...   
12787             what plane is demonstrated?   
12788   was iv contrast given to the patient?   
12789  what abnormality is seen in the image?   
12790  what abnormality is seen in the image?   
12791            what is abnormal in the mri?   

                                                answer_a  \
0                                 vascular and lymphatic   
1                                                  axial   
2                                                  axial   
3                              us-d - doppler ultrasound   
4                                                     no   
...                                                  ...   
12787                                            rickets   
12788  posterior reversible encephalopathy syndrome (...   
12789                                              axial   
12790                                              axial   
12791                                                yes   

                                                answer_b  labels_a  labels_b  \
0                                                     ap        18        13   
1                              us-d - doppler ultrasound         0        55   
2                                         an - angiogram         0        25   
3                                 vascular and lymphatic        55        18   
4                              lung, mediastinum, pleura         2         9   
...                                                  ...       ...       ...   
12787                                            coronal        87         7   
12788                                                yes        94         3   
12789  importance of recognizing vascular anatomy in ...         0       177   
12790  aneurysmal dilation of the vein of galen secon...         0       177   
12791 

In [46]:
mixed_train_df_data.shape

(12792, 9)

In [47]:
len(image_feat)

12780

In [48]:
pickle_name='mixed_train_dataset_df'
mixed_train_df_data.to_pickle(path_output_chd+'/train_dataset_pickle/'+pickle_name+'.pkl')

In [49]:
mixed_train_df_data.to_csv('mixed_train_df.csv', header=True, index=False)

In [50]:
with open(path_output_chd+'/train_dataset_pickle/mixed-train-image-feature.pickle', 'wb') as f:

    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)

In [51]:
# image_feat = {}

# for index, row in mixed_df_data():
#     print(row['c1'], row['c2'])
